In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import DataLoader, Dataset
import numpy as np
import os
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, average_precision_score, matthews_corrcoef
from sklearn.metrics import average_precision_score, coverage_error, label_ranking_loss, hamming_loss,zero_one_loss
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')


In [2]:
class RNADataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
 
    def __len__(self):
        # Return the number of samples in the dataset
        return len(self.features)
    
    def __getitem__(self, idx):
        # Retrieve the feature and label at the specified index
        feature_tensor = torch.tensor(self.features[idx], dtype=torch.float32)
        label_tensor = torch.tensor(self.labels[idx], dtype=torch.float32)
        return feature_tensor, label_tensor

In [3]:
def load_data(file_paths, label_path):
    features = []
    for file in file_paths:
        data = pd.read_csv(file).values

        # Normalize each feature set independently
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(data)  

        # Convert the scaled data to torch.tensor and append to the features list
        features.append(torch.tensor(scaled_data, dtype=torch.float32))

    all_features = torch.cat(features, dim=1)  # Concatenate along the columns

    labels = torch.tensor(pd.read_csv(label_path, header=None).values, dtype=torch.float32)

    return all_features, labels


In [4]:
def load_data(file_paths, label_path):
    features = []
    for file in file_paths:
        # Read the data from each file and convert to numpy array
        data = pd.read_csv(file).values

        # Normalize each feature set independently
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(data)  # Apply normalization

        # Convert the scaled data to torch.tensor and append to the features list
        features.append(torch.tensor(scaled_data, dtype=torch.float32))

    # Concatenate all feature sets along the feature dimension (columns)
    all_features = torch.cat(features, dim=1)  # Concatenate along the columns

    # Read the labels from the label file and convert to tensor
    labels = torch.tensor(pd.read_csv(label_path, header=None).values, dtype=torch.float32)

    return all_features, labels


In [ ]:
# Define the TextCNN module
class TextCNN(nn.Module):
    def __init__(self, out_channels_1, out_channels_2, out_channels_3):
        super(TextCNN, self).__init__()
        # Define three different convolution layers with different kernel sizes
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=out_channels_1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=out_channels_2, kernel_size=4, padding=1)
        self.conv3 = nn.Conv1d(in_channels=1, out_channels=out_channels_3, kernel_size=5, padding=1)
        # Define a pooling layer to reduce the output size to 1 using AdaptiveMaxPool1d
        self.pool = nn.AdaptiveMaxPool1d(1)
        # Define a Dropout layer for regularization to prevent overfitting
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x1 = F.relu(self.conv1(x))
        x2 = F.relu(self.conv2(x))
        x3 = F.relu(self.conv3(x))
        # Apply pooling to each convolutional output and remove the last dimension
        x1 = self.pool(x1).squeeze(-1)
        x2 = self.pool(x2).squeeze(-1)
        x3 = self.pool(x3).squeeze(-1)
        # Concatenate the outputs of the three convolutions
        x = torch.cat([x1, x2, x3], dim=1)
        x = self.dropout(x)
        return x

# Define the BiLSTM module
class BiLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=2):
        super(BiLSTM, self).__init__()
        # Define a Bi-directional LSTM layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.3)  
    
    def forward(self, x):
        # Forward pass through the LSTM
        output, _ = self.lstm(x)
        output = self.dropout(output)
        return output[:, -1, :]  

# Define the FeatureModule for handling individual features
class FeatureModule(nn.Module):
    def __init__(self, input_dim, out_channels, bilstm_hidden_dim):
        super(FeatureModule, self).__init__()
        # Initialize the TextCNN and BiLSTM modules
        self.textcnn = TextCNN(out_channels[0], out_channels[1], out_channels[2])
        self.bilstm = BiLSTM(input_dim, bilstm_hidden_dim)
        # Calculate the total dimension after concatenating the outputs
        total_dim = sum(out_channels) + 2 * bilstm_hidden_dim
        # Define the multi-head attention mechanism
        self.attention = nn.MultiheadAttention(embed_dim=total_dim, num_heads=8, batch_first=True)
        # Define the fully connected layer for classification
        self.fc = nn.Linear(total_dim, 9)
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        # Pass the input through the TextCNN module
        x_textcnn = self.textcnn(x.unsqueeze(1))
        
        # Pass the input through the BiLSTM module
        x_bilstm = self.bilstm(x.unsqueeze(1))
        
        # Concatenate the outputs of TextCNN and BiLSTM
        x_concat = torch.cat([x_textcnn, x_bilstm], dim=1).unsqueeze(1)
        
        # Apply multi-head attention to the concatenated features
        x_attention, _ = self.attention(x_concat, x_concat, x_concat)
        # Apply Dropout to the attention output
        x_attention = self.dropout(x_attention)
        x_fc = self.fc(x_attention.squeeze(1))
        return x_fc


# Define the mRSubLoc model
class mRSubLoc(nn.Module):
    def __init__(self, bilstm_hidden_dim):
        super(mRSubLoc, self).__init__()
        # Initialize feature modules for each type of feature (One-hot, Word2Vec, RNAErnie)
        self.onehot_feature_module = FeatureModule(280, [16, 32, 64], bilstm_hidden_dim)
        self.word2vec_feature_module = FeatureModule(768, [32, 64, 128], bilstm_hidden_dim)
        self.rna_ernie_feature_module = FeatureModule(768, [32, 64, 128], bilstm_hidden_dim)
        
        self.mlp = nn.Sequential(
            nn.Linear(27 , 18),  # First fully connected layer
            nn.ReLU(),  
            nn.Dropout(0.5),  # Dropout layer for regularization
            nn.Linear(18, 9)   
        )

    def forward(self, x):
        onehot_features = x[:, :280]
        word2vec_features = x[:, 280:1048]
        rna_ernie_features = x[:, 1048:]
        
        # Process each feature set through its corresponding feature module
        onehot_prob = torch.sigmoid(self.onehot_feature_module(onehot_features))
        word2vec_prob = torch.sigmoid(self.word2vec_feature_module(word2vec_features))
        rna_ernie_prob = torch.sigmoid(self.rna_ernie_feature_module(rna_ernie_features))
        
        # Concatenate the outputs of all feature modules
        new_features = torch.cat([onehot_prob, word2vec_prob, rna_ernie_prob])
        
        # Pass the concatenated features through the MLP for final classification
        final_output = self.mlp(new_features)
        return final_output


In [5]:
def calculate_metrics(L, L_pred):
    
    n, m = L.shape  # n: number of samples, m: number of labels

    # Aiming 
    aiming = 0
    for v in range(n):
        intersection = 0
        for h in range(m):
            if L_pred[v, h] == 1 and L[v, h] == 1:
                intersection += 1
        if sum(L_pred[v]) == 0: 
            continue
        aiming += intersection / sum(L_pred[v])  
    aiming /= n

    # Accuracy
    accuracy = 0
    for v in range(n):
        intersection = 0
        union = 0
        for h in range(m):
            if L_pred[v, h] == 1 or L[v, h] == 1:
                union += 1
            if L_pred[v, h] == 1 and L[v, h] == 1:
                intersection += 1
        if union == 0:  
            continue
        accuracy += intersection / union  
    accuracy /= n

    # Coverage 
    coverage = 0
    for v in range(n):
        intersection = 0
        for h in range(m):
            if L_pred[v, h] == 1 and L[v, h] == 1:
                intersection += 1
        if sum(L[v]) == 0:  
            continue
        coverage += intersection / sum(L[v])  
    coverage /= n

    # AbsoluteTrue
    absolute_true = 0
    for v in range(n):
        if list(L_pred[v]) == list(L[v]):  
            absolute_true += 1
    absolute_true /= n

    # AbsoluteFalse 
    absolute_false = 0
    for v in range(n):
        intersection = 0
        union = 0
        for h in range(m):
            if L_pred[v, h] == 1 or L[v, h] == 1:
                union += 1
            if L_pred[v, h] == 1 and L[v, h] == 1:
                intersection += 1
        absolute_false += (union - intersection) / m  
    absolute_false /= n

    return aiming, coverage, accuracy, absolute_true, absolute_false


In [6]:
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, device):
    best_hamming_loss = float('inf')  # Initialize best Hamming loss as infinity
    best_model_state = None  

    y_true_list = []  
    y_pred_list = []  

    # Training mode
    model.train()  
    for features, labels in train_loader:
        features = features.to(device)  
        labels = labels.to(device)  

        optimizer.zero_grad()  # Clear previous gradients
        outputs = model(features)  # Forward pass through the model
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the model parameters

    # Validation mode
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient calculation during validation
        for features, labels in val_loader:
            features = features.to(device)  
            labels = labels.to(device) 
            outputs = model(features)  
            preds = torch.sigmoid(outputs).round()  # Apply sigmoid and round to get binary predictions
            y_true_list.append(labels.cpu().numpy())  # Append true labels to the list
            y_pred_list.append(preds.cpu().numpy())  # Append predicted labels to the list

    # Stack true and predicted labels from the lists to form arrays
    y_true = np.vstack(y_true_list)
    y_pred = np.vstack(y_pred_list)

    # Calculate evaluation metrics
    aiming_value, coverage_value, accuracy_value, absolute_true_value, absolute_false_value = calculate_metrics(y_true, y_pred)

    return aiming_value, coverage_value, accuracy_value, absolute_true_value, absolute_false_value


In [7]:
def main():
    feature_files = ['../Data/TrainingData_Onehot.csv',
                     '../Data/TrainingData_word2vec.csv',
                     '../Data/TrainingData_RNAErnie.csv']
    label_file = '../Data/TrainingLabel.csv'
    bilstm_hidden_dim = 64  # Hidden dimension for BiLSTM
    batch_size = 32  
    learning_rate = 0.00001  
    k_folds = 5  
    epochs = 100  
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

    # Load data
    features, labels = load_data(feature_files, label_file)
    features, labels = shuffle(features, labels, random_state=43)  

    # Initialize K-Fold cross-validation
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=43)

    global_acc = 0.0  # Variable to track the best global accuracy
    global_best_model = None 

    # Dictionary to store metrics across all folds
    fold_metrics_all = {
        "aiming": [],
        "coverage": [],
        "accuracy": [],
        "absolute_true": [],
        "absolute_false": []
    }

    # K-Fold Cross-Validation outer loop
    for fold, (train_idx, val_idx) in enumerate(kf.split(labels), 1):
        
        # Split the data into training and validation sets
        train_features = features[train_idx]
        val_features = features[val_idx] 
        train_labels = labels[train_idx]
        val_labels = labels[val_idx]

        # DataLoader for training and validation sets
        train_dataset = RNADataset(train_features, train_labels)
        val_dataset = RNADataset(val_features, val_labels)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        # Instantiate the model
        model = mRSubLoc(
            bilstm_hidden_dim=bilstm_hidden_dim
        ).to(device)

        # Loss function and optimizer
        criterion = nn.BCEWithLogitsLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        # Dictionary to store metrics for each fold
        fold_metrics = {
            "aiming": [],
            "coverage": [],
            "accuracy": [],
            "absolute_true": [],
            "absolute_false": []
        }

        # Loop over epochs for the current fold
        for epoch in range(epochs):
            # Train and validate the model
            aiming_value, coverage_value, accuracy_value, absolute_true_value, absolute_false_value = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, device)

            # Print metrics for the current fold and epoch
            print(f"Fold {fold}, Epoch {epoch + 1} : "
                  f"Aiming: {aiming_value:.4f}, "
                  f"Coverage: {coverage_value:.4f}, "
                  f"Accuracy: {accuracy_value:.4f}, "
                  f"Absolute True: {absolute_true_value:.4f}, "
                  f"Absolute False: {absolute_false_value:.4f}")

            # Store metrics for each epoch in the current fold
            fold_metrics["aiming"].append(aiming_value)
            fold_metrics["coverage"].append(coverage_value)
            fold_metrics["accuracy"].append(accuracy_value)
            fold_metrics["absolute_true"].append(absolute_true_value)
            fold_metrics["absolute_false"].append(absolute_false_value)

            
            if accuracy_value > global_acc: 
                global_acc = accuracy_value
                global_best_model = model.state_dict()

        # Store the metrics for the current fold in the global dictionary
        for metric in fold_metrics_all:
            fold_metrics_all[metric].append(np.mean(fold_metrics[metric]))

    # Calculate the average metrics across all folds
    avg_fold_metrics = {metric: np.mean(values) for metric, values in fold_metrics_all.items()}
    print(f"\nAverage metrics across all folds: "
          f"Aiming: {avg_fold_metrics['aiming']:.4f}, "
          f"Coverage: {avg_fold_metrics['coverage']:.4f}, "
          f"Accuracy: {avg_fold_metrics['accuracy']:.4f}, "
          f"Absolute True: {avg_fold_metrics['absolute_true']:.4f}, "
          f"Absolute False: {avg_fold_metrics['absolute_false']:.4f}")

Fold 1, Epoch 1 : Aiming: 0.0000, Coverage: 0.0000, Accuracy: 0.0000, Absolute True: 0.0000,Absolute False: 0.3332
Fold 1, Epoch 2 : Aiming: 0.0000, Coverage: 0.0000, Accuracy: 0.0000, Absolute True: 0.0000,Absolute False: 0.3332
Fold 1, Epoch 3 : Aiming: 0.0000, Coverage: 0.0000, Accuracy: 0.0000, Absolute True: 0.0000,Absolute False: 0.3332
Fold 1, Epoch 4 : Aiming: 0.3156, Coverage: 0.0494, Accuracy: 0.0494, Absolute True: 0.0000,Absolute False: 0.2981
Fold 1, Epoch 5 : Aiming: 0.7919, Coverage: 0.4255, Accuracy: 0.4255, Absolute True: 0.2963,Absolute False: 0.2683
Fold 1, Epoch 6 : Aiming: 0.7919, Coverage: 0.4255, Accuracy: 0.4255, Absolute True: 0.2963,Absolute False: 0.2683
Fold 1, Epoch 7 : Aiming: 0.7919, Coverage: 0.4255, Accuracy: 0.4255, Absolute True: 0.2963,Absolute False: 0.2683
Fold 1, Epoch 8 : Aiming: 0.7919, Coverage: 0.4255, Accuracy: 0.4255, Absolute True: 0.2963,Absolute False: 0.2683
Fold 1, Epoch 9 : Aiming: 0.7919, Coverage: 0.4255, Accuracy: 0.4255, Absolute T

Fold 1, Epoch 72 : Aiming: 0.7593, Coverage: 0.6724, Accuracy: 0.6505, Absolute True: 0.3626,Absolute False: 0.1099
Fold 1, Epoch 73 : Aiming: 0.7588, Coverage: 0.6722, Accuracy: 0.6502, Absolute True: 0.3626,Absolute False: 0.1100
Fold 1, Epoch 74 : Aiming: 0.7579, Coverage: 0.6729, Accuracy: 0.6501, Absolute True: 0.3620,Absolute False: 0.1100
Fold 1, Epoch 75 : Aiming: 0.7576, Coverage: 0.6747, Accuracy: 0.6510, Absolute True: 0.3635,Absolute False: 0.1097
Fold 1, Epoch 76 : Aiming: 0.7600, Coverage: 0.6716, Accuracy: 0.6505, Absolute True: 0.3631,Absolute False: 0.1100
Fold 1, Epoch 77 : Aiming: 0.7608, Coverage: 0.6707, Accuracy: 0.6503, Absolute True: 0.3635,Absolute False: 0.1098
Fold 1, Epoch 78 : Aiming: 0.7584, Coverage: 0.6719, Accuracy: 0.6499, Absolute True: 0.3629,Absolute False: 0.1100
Fold 1, Epoch 79 : Aiming: 0.7573, Coverage: 0.6733, Accuracy: 0.6503, Absolute True: 0.3633,Absolute False: 0.1100
Fold 1, Epoch 80 : Aiming: 0.7572, Coverage: 0.6730, Accuracy: 0.6499, A

Fold 2, Epoch 43 : Aiming: 0.8399, Coverage: 0.6693, Accuracy: 0.6024, Absolute True: 0.2952,Absolute False: 0.1774
Fold 2, Epoch 44 : Aiming: 0.8393, Coverage: 0.6809, Accuracy: 0.6110, Absolute True: 0.2989,Absolute False: 0.1731
Fold 2, Epoch 45 : Aiming: 0.8358, Coverage: 0.6897, Accuracy: 0.6178, Absolute True: 0.3013,Absolute False: 0.1697
Fold 2, Epoch 46 : Aiming: 0.8339, Coverage: 0.6792, Accuracy: 0.6118, Absolute True: 0.2998,Absolute False: 0.1720
Fold 2, Epoch 47 : Aiming: 0.8320, Coverage: 0.6850, Accuracy: 0.6230, Absolute True: 0.3031,Absolute False: 0.1651
Fold 2, Epoch 48 : Aiming: 0.8258, Coverage: 0.6669, Accuracy: 0.6183, Absolute True: 0.3024,Absolute False: 0.1627
Fold 2, Epoch 49 : Aiming: 0.8238, Coverage: 0.6629, Accuracy: 0.6175, Absolute True: 0.3029,Absolute False: 0.1617
Fold 2, Epoch 50 : Aiming: 0.8273, Coverage: 0.6773, Accuracy: 0.6274, Absolute True: 0.3057,Absolute False: 0.1585
Fold 2, Epoch 51 : Aiming: 0.8283, Coverage: 0.6860, Accuracy: 0.6351, A

Fold 3, Epoch 14 : Aiming: 0.7514, Coverage: 0.6663, Accuracy: 0.5715, Absolute True: 0.2275,Absolute False: 0.1708
Fold 3, Epoch 15 : Aiming: 0.7563, Coverage: 0.6749, Accuracy: 0.5899, Absolute True: 0.2571,Absolute False: 0.1568
Fold 3, Epoch 16 : Aiming: 0.7594, Coverage: 0.6779, Accuracy: 0.6001, Absolute True: 0.2695,Absolute False: 0.1496
Fold 3, Epoch 17 : Aiming: 0.7657, Coverage: 0.6716, Accuracy: 0.6065, Absolute True: 0.2843,Absolute False: 0.1453
Fold 3, Epoch 18 : Aiming: 0.7677, Coverage: 0.6882, Accuracy: 0.6296, Absolute True: 0.2998,Absolute False: 0.1293
Fold 3, Epoch 19 : Aiming: 0.7732, Coverage: 0.6951, Accuracy: 0.6505, Absolute True: 0.3333,Absolute False: 0.1132
Fold 3, Epoch 20 : Aiming: 0.7816, Coverage: 0.6772, Accuracy: 0.6556, Absolute True: 0.3597,Absolute False: 0.1100
Fold 3, Epoch 21 : Aiming: 0.7844, Coverage: 0.6677, Accuracy: 0.6550, Absolute True: 0.3666,Absolute False: 0.1095
Fold 3, Epoch 22 : Aiming: 0.7841, Coverage: 0.6690, Accuracy: 0.6559, A

Fold 3, Epoch 85 : Aiming: 0.7709, Coverage: 0.6841, Accuracy: 0.6640, Absolute True: 0.3815,Absolute False: 0.1070
Fold 3, Epoch 86 : Aiming: 0.7717, Coverage: 0.6825, Accuracy: 0.6632, Absolute True: 0.3804,Absolute False: 0.1072
Fold 3, Epoch 87 : Aiming: 0.7730, Coverage: 0.6807, Accuracy: 0.6626, Absolute True: 0.3788,Absolute False: 0.1072
Fold 3, Epoch 88 : Aiming: 0.7715, Coverage: 0.6821, Accuracy: 0.6626, Absolute True: 0.3791,Absolute False: 0.1073
Fold 3, Epoch 89 : Aiming: 0.7721, Coverage: 0.6812, Accuracy: 0.6627, Absolute True: 0.3795,Absolute False: 0.1073
Fold 3, Epoch 90 : Aiming: 0.7674, Coverage: 0.6847, Accuracy: 0.6626, Absolute True: 0.3801,Absolute False: 0.1076
Fold 3, Epoch 91 : Aiming: 0.7701, Coverage: 0.6829, Accuracy: 0.6628, Absolute True: 0.3810,Absolute False: 0.1075
Fold 3, Epoch 92 : Aiming: 0.7696, Coverage: 0.6829, Accuracy: 0.6623, Absolute True: 0.3791,Absolute False: 0.1076
Fold 3, Epoch 93 : Aiming: 0.7679, Coverage: 0.6845, Accuracy: 0.6625, A

Fold 4, Epoch 56 : Aiming: 0.7464, Coverage: 0.6854, Accuracy: 0.6513, Absolute True: 0.3732,Absolute False: 0.1106
Fold 4, Epoch 57 : Aiming: 0.7448, Coverage: 0.6875, Accuracy: 0.6519, Absolute True: 0.3725,Absolute False: 0.1107
Fold 4, Epoch 58 : Aiming: 0.7463, Coverage: 0.6867, Accuracy: 0.6526, Absolute True: 0.3767,Absolute False: 0.1099
Fold 4, Epoch 59 : Aiming: 0.7445, Coverage: 0.6884, Accuracy: 0.6528, Absolute True: 0.3749,Absolute False: 0.1103
Fold 4, Epoch 60 : Aiming: 0.7448, Coverage: 0.6883, Accuracy: 0.6531, Absolute True: 0.3767,Absolute False: 0.1099
Fold 4, Epoch 61 : Aiming: 0.7455, Coverage: 0.6880, Accuracy: 0.6538, Absolute True: 0.3799,Absolute False: 0.1092
Fold 4, Epoch 62 : Aiming: 0.7476, Coverage: 0.6854, Accuracy: 0.6534, Absolute True: 0.3812,Absolute False: 0.1089
Fold 4, Epoch 63 : Aiming: 0.7467, Coverage: 0.6871, Accuracy: 0.6538, Absolute True: 0.3806,Absolute False: 0.1090
Fold 4, Epoch 64 : Aiming: 0.7452, Coverage: 0.6892, Accuracy: 0.6549, A

Fold 5, Epoch 27 : Aiming: 0.7815, Coverage: 0.6385, Accuracy: 0.6278, Absolute True: 0.3314,Absolute False: 0.1279
Fold 5, Epoch 28 : Aiming: 0.7822, Coverage: 0.6444, Accuracy: 0.6336, Absolute True: 0.3412,Absolute False: 0.1235
Fold 5, Epoch 29 : Aiming: 0.7827, Coverage: 0.6466, Accuracy: 0.6358, Absolute True: 0.3455,Absolute False: 0.1217
Fold 5, Epoch 30 : Aiming: 0.7823, Coverage: 0.6544, Accuracy: 0.6428, Absolute True: 0.3523,Absolute False: 0.1168
Fold 5, Epoch 31 : Aiming: 0.7820, Coverage: 0.6576, Accuracy: 0.6455, Absolute True: 0.3553,Absolute False: 0.1149
Fold 5, Epoch 32 : Aiming: 0.7820, Coverage: 0.6621, Accuracy: 0.6497, Absolute True: 0.3616,Absolute False: 0.1119
Fold 5, Epoch 33 : Aiming: 0.7826, Coverage: 0.6643, Accuracy: 0.6521, Absolute True: 0.3636,Absolute False: 0.1099
Fold 5, Epoch 34 : Aiming: 0.7823, Coverage: 0.6655, Accuracy: 0.6530, Absolute True: 0.3642,Absolute False: 0.1096
Fold 5, Epoch 35 : Aiming: 0.7826, Coverage: 0.6651, Accuracy: 0.6528, A

Fold 5, Epoch 98 : Aiming: 0.7801, Coverage: 0.6939, Accuracy: 0.6697, Absolute True: 0.3749,Absolute False: 0.0972
Fold 5, Epoch 99 : Aiming: 0.7809, Coverage: 0.6948, Accuracy: 0.6701, Absolute True: 0.3747,Absolute False: 0.0969
Fold 5, Epoch 100 : Aiming: 0.7820, Coverage: 0.6947, Accuracy: 0.6710, Absolute True: 0.3764,Absolute False: 0.0967

Average metrics across all folds: Aiming: 0.7569, Coverage: 0.6579, Accuracy: 0.6092, Absolute True: 0.3205, Absolute False: 0.1415
Saving model to models\RNAErnie+Onehot+Word2Vec.pth
Best model saved with Acc: 0.6829
